# Part II - (Explanatory data analysis of Prosper loan dataset)
## by (Raman Precious)




## Investigation Overview


> The overall goal of this presentation is share with people the importance of rebranding when a company is not doing so well, using the Prosper organization as a case study, as well as exa,mining the realtionship between Prosper Rating and Lender yield. The key insights reveal by the several analysis of the given dataset are:

- There was a dip in 2009 that indicates a 'pause period' where the company rebranded itself and raised the bar of crediting goals and visions.

- The number of defaulted loans pre-revolution of the company is higher than that of  post-revolution. This shows that the decision to revolutionize the company was a success. There is however a concerning observation of increased defaulted loan from 2012. This should be thoroughly checked.

- High risk loans have a higher lender yield. This is profitable for lenders, as borrowers with poor credit rating will have higher APR which returns higher yields. Also, there are many defaulted loans in the high risk category, thereby proving that the prosper rating system for loan approval is somewhat accurate.



## Dataset Overview

> I investigated the Prosper loan dataset provided by Prosper for insights and visualizations that would shed light on the progress of the company and their operations. This prosper data set contains 113,937 loans with 81 variables on each loan, including loan amount, borrowers' rate (or interest rate), current loan status, borrowers' income, and many others.

Data dictionary for better understanding of the variables are in this link: https://docs.google.com/spreadsheets/d/1gDyi_L4UvIrLTEC6Wri5nbaMmkGmLQBk-Yx3z0XDEtI/edit#gid=0.

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")

In [2]:
loan_df = pd.read_csv('prosperLoanData.csv')


FileNotFoundError: File b'prosperLoanData.csv' does not exist

> Note that the above cells have been set as "Skip"-type slides. That means
that when the notebook is rendered as http slides, those cells won't show up.

In [ ]:
def nullV() :  
    y = loan_df.isnull().sum()
    for i in range(len(y)) :
        if y[i] != 0 :
            x= y[i] 
            col = loan_df.columns[i]
            z= (x/loan_df.shape[0])*100
            print(str(i) + " - " + col + " - " + str(x) + " nulls -"+ str(round(z,2)) +"%")
nullV()    

In [ ]:
loan_df.drop(columns=["TotalProsperLoans","TotalProsperPaymentsBilled",
                 "ProsperPaymentsLessThanOneMonthLate","ProsperPaymentsOneMonthPlusLate","ProsperPrincipalBorrowed",
                "ProsperPrincipalOutstanding","ScorexChangeAtTimeOfListing","LoanFirstDefaultedCycleNumber","OnTimeProsperPayments"],inplace= True )

loan_df.drop(columns = ['ListingKey', 'ListingNumber', 'GroupKey', 'LoanKey', 'LoanNumber', 'MemberKey'], inplace = True)

In [ ]:
borrower_fees = loan_df['BorrowerAPR'] - loan_df['BorrowerRate']
fees_mean = borrower_fees.mean()
loan_df['BorrowerAPR'].fillna((loan_df['BorrowerRate'] + fees_mean), inplace = True)


DTIR = loan_df['DebtToIncomeRatio'].mean()
loan_df['DebtToIncomeRatio'].fillna(DTIR, inplace = True)

loan_df.dropna(subset=[ 'Occupation','BorrowerState','CurrentDelinquencies',"CreditScoreRangeLower", 'OpenCreditLines'], inplace=True)

In [ ]:
# convert columns from 'object' datatype to 'datetime'
loan_df['ListingCreationDate'] = pd.to_datetime(loan_df['ListingCreationDate'])
loan_df['LoanOriginationDate'] = pd.to_datetime(loan_df['LoanOriginationDate'])
loan_df['ClosedDate'] = pd.to_datetime(loan_df['ClosedDate'])

# split variables into day, month and year
loan_df['ListingDay'] = loan_df['ListingCreationDate'].dt.day
loan_df['ListingMonth'] = loan_df['ListingCreationDate'].dt.month_name()
loan_df['ListingYear'] = loan_df['ListingCreationDate'].dt.year
loan_df['OriginationDay'] = loan_df['LoanOriginationDate'].dt.day
loan_df['OriginationMonth'] = loan_df['LoanOriginationDate'].dt.month_name()
loan_df['OriginationYear'] = loan_df['LoanOriginationDate'].dt.year
loan_df['ClosedDay'] = loan_df['ClosedDate'].dt.day
loan_df['ClosedMonth'] = loan_df['ClosedDate'].dt.month_name()
loan_df['ClosedYear'] = loan_df['ClosedDate'].dt.year
                                                        

In [ ]:
loan_df['ProsperRating (numeric)'].replace(to_replace = [0,1,2,3,4,5,6,7], value = ['N/A', 'HR', 'E', 'D', 'C','B', 'A', 'AA'], inplace = True)
loan_df.rename(columns = {'ProsperRating (numeric)':'ProsperRating'}, inplace = True)

loan_df['ListingCategory (numeric)'].replace(to_replace=[0,1,2,3,4,5,6,7,8, 9, 10,11,12,13,14,15,16,17,18,19,20],
                                        value=['Not Available','Debt Consolidation','Home Improvement','Business','Personal Loan',
                                               'Student Use','Auto','Other','Baby&Adoption','Boat','Cosmetic Procedure',
                                               'Engagement Ring','Green Loans','Household Expenses','Large Purchases',
                                               'Medical/Dental','Motorcycle,','RV','Taxes','Vacation','Wedding Loans'],inplace=True)
loan_df.rename(columns = {'ListingCategory (numeric)':'ListingCategory'}, inplace = True)

In [ ]:
ordinal_dict = {'ProsperRating': ['N/A','HR','E','D','C', 'B', 'A', 'AA']}
for var in ordinal_dict:
    ordered_var = pd.api.types.CategoricalDtype(ordered = True, categories = ordinal_dict[var])
    loan_df[var] = loan_df[var].astype(ordered_var)
    
loan_df.drop(columns = ['ListingCreationDate','StatedMonthlyIncome','MonthlyLoanPayment','LoanOriginationDate', 'ClosedDate','AmountDelinquent', 'LoanMonthsSinceOrigination', 'LoanCurrentDaysDelinquent', 'FirstRecordedCreditLine', 'EstimatedEffectiveYield', 'EstimatedLoss',
                       'EstimatedReturn', 'ProsperRating (Alpha)', 'CurrentlyInGroup', 'DateCreditPulled',
                       'DelinquenciesLast7Years', 'PublicRecordsLast10Years', 'BankcardUtilization',
                       'PublicRecordsLast12Months', 'ProsperScore', 'TotalTrades', 'TradesNeverDelinquent (percentage)',
                       'TradesOpenedLast6Months', 'IncomeVerifiable', 'LP_CustomerPayments', 'LP_CustomerPrincipalPayments',
                       'LP_InterestandFees', 'LP_ServiceFees', 'LP_CollectionFees', 'LP_GrossPrincipalLoss', 'LP_NetPrincipalLoss',
                       'LP_NonPrincipalRecoverypayments', 'PercentFunded', 'Recommendations', 'InvestmentFromFriendsCount',
                       'InvestmentFromFriendsAmount', 'RevolvingCreditBalance','Investors', 'EmploymentStatusDuration','OpenCreditLines',
                       'OpenRevolvingAccounts','AvailableBankcardCredit','OpenRevolvingMonthlyPayment','TotalCreditLinespast7years', 'TotalInquiries',  'InquiriesLast6Months'], inplace = True )

In [ ]:
loan_df['Occupation'] = loan_df['Occupation'].astype('category')
loan_df['BorrowerState'] = loan_df['BorrowerState'].astype('category')
loan_df['LoanStatus'] = loan_df['LoanStatus'].astype('category')
loan_df['ListingCategory'] = loan_df['ListingCategory'].astype('category')
loan_df['EmploymentStatus'] = loan_df['EmploymentStatus'].astype('category')


loan_df['CreditScore'] = (loan_df['CreditScoreRangeUpper'] + loan_df['CreditScoreRangeLower'])/2
loan_df.drop(columns =['CreditScoreRangeLower','CreditScoreRangeUpper'], inplace = True)

In [ ]:
loan_df['LoanOriginationQuarter'] = loan_df['LoanOriginationQuarter'].str.split(' ').str[0]

loan_df['LoanOriginationQuarter'] = loan_df['OriginationYear'].map(str) +  ' ' + loan_df['LoanOriginationQuarter'].map(str)

loan_df["LoanStatus"]= loan_df["LoanStatus"].replace({"FinalPaymentInProgress": "Current", "Past Due (1-15 days)": "Past Due", 'Cancelled': 'Defaulted', 'Chargedoff' : 'Defaulted', 'Past Due (16-30 days)': 'Past Due', 'Past Due (31-60 days)': 'Past Due','Past Due (61-90 days)': 'Past Due', 'Past Due (91-120 days)': 'Past Due', 'Past Due (>120 days)': 'Past Due'})


## (Visualization 1)

> There was a dip in 2009 that indicates a 'pause period' where the company rebranded itself and raised the bar of crediting goals and visions.



In [ ]:
# create a time series

d = loan_df.groupby(['LoanOriginationQuarter','LoanStatus',]).size().unstack()
plt.figure(figsize = [10, 8])
d.plot(kind = 'bar', stacked = True, legend = True)
plt.ylabel('counts')
plt.title('Time series to show relationship between Quarter and Loan status group');

{
    "tags": [
        "to_remove"
    ],
    "slideshow": {
        "slide_type": "Sub-Slide"
    }
}

> This chart shows the performance of the loans since their commencement. It is revealed that a large number of the loans requested are either completed or current with a low level of defaulted and past due loans. It reveals the productivity trend of the company over the years. The company also began to issue lots of loan in recent times which signifies a high productivity. Some of the unusual distributions are the absence of the 2009 quarter one and non-productivity in quarter two. This is sure to be the cause of the dip in 2009 and indicates a 'pause period' where the company was  rebranded, stategies were inculcated, and the bar of high crediting goals and visions was raised

## (Visualization 2)

> The number of defaulted loans pre-revolution of the company is higher than that of post-revolution. This shows that the decision to revolutionize the company was a success. There is however a concerning observation of increased defaulted loan from 2012. This should be thoroughly checked.

In [ ]:
# create time series
b = loan_df[loan_df['LoanStatus'] == 'Defaulted']

bb = b['LoanOriginationQuarter'].value_counts().rename_axis('Quarters').reset_index(name='counts').sort_values('Quarters')
figsize = [10,5]
f, ax = plt.subplots(1, 1, figsize=figsize)

x_col= 'Quarters'
y_col = 'counts'

sb.pointplot(ax=ax, x=x_col,y=y_col,data=bb)

plt.tick_params(axis='x', rotation=90)
plt.title('Time series of defaulted loan per quarter');

{
    "tags": [
        "to_remove"
    ],
    "slideshow": {
        "slide_type": "Sub-Slide"
    }
}

> A drastic reduction in default payments is noticed between 2008 - 2009 as well as towards 2013. This plot was chosen because it is straightforward and delivers its message precisely. Prior to 2009, we can see that the rate generally hovered around 1000 to 1400 loans, which is significantly high and a clear area for improvement. Prosper stopped making new loans between the fourth quarter of 2008 and the third quarter of 2009. That explains the sharp drop during that time period. They maintained that low default rate throughout 2010, when they restarted their service with new policies. During 2011, it returned to around 30% and remained there until 2012. It is important to note that from 2011 onwards, there are still loans in operation, and we cannot draw conclusions based on the data. The decrease in 2013 is due to the fact that the majority of loans at the time are either current or in the final stages of repayment.

## (Visualization 3)

> High risk loans have a higher lender yield. This is profitable for lenders, as borrowers with poor credit rating will have higher APR which returns higher yields. Also, there are many defaulted loans in the high risk category, thereby proving that the prosper rating system for loan approval is somewhat accurate.


In [ ]:
g = sb.FacetGrid(data = loan_df, col = 'LoanStatus', hue = 'ProsperRating', hue_order = ['AA', 'A', 'B', 'C', 'D', 'E', 'HR'],aspect = 1.5,palette =  'viridis_r' , size = 5, col_wrap = 2)
g.map(sb.regplot, 'LoanOriginalAmount', 'LenderYield', fit_reg = False, x_jitter = 0.04)
plt.legend(bbox_to_anchor=[1.15,0.7], title= 'Risk rating');

{
    "tags": [
        "to_remove"
    ],
    "slideshow": {
        "slide_type": "Sub-Slide"
    }
}

> Defaulted category only has loans of 25,000 dollars and below. 

>There are not many loans above 25,000 dollars. This could be because borrowers with a riskier rating got approved for lower loan amounts, which makes sense considering the fact that they may be under heavy financial pressure. 

>In the Current category,there are much more loans being taken past the 25,000 dollar mark and even past the 30,000 dollars mark and veering towards the maximum of 35000 dollars.

>It is also observed that high risk loans  have a higher lender yield. This is understandable as borrowers with poor credit rating will have higher APR which return higher yields. 

>The plot also shows that many defaulted loans are in the high risk category. This proves that the prosper rating system for loan approval is somewhat accurate.

### Generate Slideshow
Once you're ready to generate your slideshow, use the `jupyter nbconvert` command to generate the HTML slide show.  

In [ ]:
# Use this command if you are running this file in local
!jupyter nbconvert <Part_II_slide_deck_template>.ipynb --to slides --post serve --no-input --no-prompt

In [ ]:

!jupyter nbconvert Part_II_slide_deck_template.ipynb --to slides --post serve --template output-toggle.tpl


> In the classroom workspace, the generated HTML slideshow will be placed in the home folder. 

> In local machines, the command above should open a tab in your web browser where you can scroll through your presentation. Sub-slides can be accessed by pressing 'down' when viewing its parent slide. Make sure you remove all of the quote-formatted guide notes like this one before you finish your presentation! At last, you can stop the Kernel. 

### Submission
If you are using classroom workspace, you can choose from the following two ways of submission:

1. **Submit from the workspace**. Make sure you have removed the example project from the /home/workspace directory. You must submit the following files:
   - Part_I_notebook.ipynb
   - Part_I_notebook.html or pdf
   - Part_II_notebook.ipynb
   - Part_I_slides.html
   - README.md
   - dataset (optional)


2. **Submit a zip file on the last page of this project lesson**. In this case, open the Jupyter terminal and run the command below to generate a ZIP file. 
```bash
zip -r my_project.zip .
```
The command abobve will ZIP every file present in your /home/workspace directory. Next, you can download the zip to your local, and follow the instructions on the last page of this project lesson.
